In [1]:
# Import the required packages
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast, os
import bambi as bmb
import pymc as pm
import arviz as az
import scipy.stats as stat
from collections import Counter
import itertools
import hssm


In [ ]:
#Set up plotting themes
sns.set_context('paper')
sns.set_style('darkgrid')
sns.set_palette('colorblind')

# Plot RTs and Accuracies

In [ ]:
# Read in data
phase = 'exposure' #Options 'exposure' or 'memory'
if phase == 'exposure':
    df_clean = pd.read_csv('df_clean_exposure.csv')
else:
    df_clean = pd.read_csv('df_clean_memory')

df_clean_grouped = df_clean.groupby(['participant', 'block', 'node type', 'condition']).mean(numeric_only=True).reset_index()

In [ ]:
# Set plot parameters
sns.set_context('paper')
sns.set(font_scale=1.5)

In [ ]:
#Specify which measure to be plotted ('accuracy' or 'rt')
measure = 'accuracy'


g = sns.catplot(col = 'node type', y = measure,  
           data = df_clean, kind = 'point', margin_titles=True, col_order = ['boundary', 'nonboundary'],#, 'new'],
                x = 'block', hue = 'condition', ci=95, hue_order=['unstructured', 'structured'])
sns.stripplot(y = measure, x = 'block', hue = 'condition', 
              data = df_clean_grouped.loc[df_clean_grouped['node type'] == 'boundary'], 
              ax=g.axes[0][0], legend=False, alpha = 0.4)
sns.stripplot(y = measure, x = 'block', hue = 'condition', 
              data = df_clean_grouped.loc[df_clean_grouped['node type'] == 'nonboundary'], 
              ax=g.axes[0][1], legend=False, alpha = 0.4)

# Uncomment the line below for plotting memory to also plot responses for new items
# sns.stripplot(y = measure, x = 'block', hue = 'condition', 
#               data = df_clean_memory_grouped.loc[df_clean_memory_grouped['node type'] == 'new'], 
#               ax=g.axes[0][2], legend=False, alpha = 0.4)

g._legend.set_title('Graph type')
g.axes[0][0].set_title('Boundary Node Items')
g.axes[0][1].set_title('Non Boundary Node Items')
# g.axes[0][2].set_title('New Items') #Uncomment for new

g.axes[0][0].set_ylabel(f'Exposure {measure} \n (proportion correct)')
g.axes[0][0].set_xlabel('Block')
g.axes[0][1].set_xlabel('Block')
# g.axes[0][2].set_xlabel('Block') #Uncomment for new


g.axes[0][0].set_xticklabels(['1', '2', '3'])
g.axes[0][1].set_xticklabels(['1', '2', '3'])
# g.axes[0][2].set_xticklabels(['1', '2', '3']) #Uncomment for new

plt.savefig(f'figures/exposure_{measure}.png', dpi = 300)


# Read stats and model outputs
Code below presents an example to save stat tables which are in the manuscript. 

Can only be run after posterior traces are created and saved in appropriate files. See 'bayesian_models.py' for details

## RT/Accuracy by exposure time

In [ ]:
samples = az.from_netcdf('model_results/exposure_nt_accuracy.nc') #Read model outputs
stat_table = az.summary(samples, filter_vars='like', var_names = ['trials', 'condition', 'trials', 'node_type', 'Intercept'], hdi_prob=.95, round_to=2) #Compute summary
stat_table[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%']].to_latex('model_results/stats/accuracy.txt', float_format="%.2f") #Save to latex table
stat_table[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%']] #Print in notebook

## DDM

Can only be run after creating model posteriors. See hssm_modelfits.py for doing so

In [ ]:
#Read (best) fit models for structured and unstructured group participants
ddm_model_samples_unstructured = az.from_netcdf(f'hssm_results/unstructured_v~acc_exp-nodetype-nodetype|ppt_a~block-|ppt_z~block-|ppt_t~|ppt.nc')
ddm_model_samples_structured = az.from_netcdf(f'hssm_results/structured_v~acc_exp-nodetype-nodetype|ppt_a~block-|ppt_z~block-|ppt_t~|ppt.nc')


In [ ]:
sns.set_palette('colorblind')
sns.set(font_scale = 1)

#Read posteriors of boundary/non boundary parameters for both participant groups
bs = ddm_model_samples_structured['posterior'].sel({'v_C(node_type)_dim': 'boundary'})['v_C(node_type)']
nbs = ddm_model_samples_structured['posterior'].sel({'v_C(node_type)_dim': 'nonboundary'})['v_C(node_type)']
bus = ddm_model_samples_unstructured['posterior'].sel({'v_C(node_type)_dim': 'boundary'})['v_C(node_type)']
nbus = ddm_model_samples_unstructured['posterior'].sel({'v_C(node_type)_dim': 'nonboundary'})['v_C(node_type)']

#Plot differences between boundary and non-boundary for both groups
sns.histplot(np.ravel(bus-nbus), alpha = .5, label = 'Unstructured Graph')
sns.histplot(np.ravel(bs - nbs), alpha = .5, label = 'Structured Graph')

plt.axvline(x = 0, ls = '--', color = 'black')
plt.text(x = np.mean(bs-nbs), y = 200, s = f'{str((np.array(np.mean(((bs-nbs)>0))))*100)[:4]}% samples > 0', weight = 'bold')
plt.text(x = np.mean(bus-nbus), y = 175, s = f'{str((np.array(np.mean(((bus-nbus)>0))))*100)[:4]}% samples > 0', weight = 'bold')
plt.legend(loc = 'upper left')
plt.xlabel('Boundary - Non Boundary \n Drift Rate Differences')
plt.tight_layout()
plt.savefig('figures/ddm_results.png', dpi = 300)

## DDM Stats

In [ ]:
#Compute stats and store summary in latex table
CONDITION = 'structured'
#Change the file name (first argument) for whichever condition these are to be computed
az.summary(ddm_model_samples_structured, var_names= '~participant', filter_vars='like', hdi_prob=.95)[['mean', 'sd', 'hdi_2.5%', 'hdi_97.5%']].to_latex(f'model_results/stats/ddm_params_{CONDITION}.txt', float_format="%.2f")


## DDM Model Comparisons

Can only be done after all candidate models are fit using hssm_modelfits file.

In [ ]:
# Specify which participant group
CONDITION = 'structured'
# Read all candidate model posteriors
ddm_model_ablock_samples = az.from_netcdf(f'hssm_results/{CONDITION}_v~acc_exp-1|ppt_a~block-|ppt_z~|ppt_t~|ppt.nc')
ddm_model_nt_ablock_samples = az.from_netcdf(f'hssm_results/{CONDITION}_v~acc_exp-nodetype-nodetype|ppt_a~block-|ppt_z~|ppt_t~|ppt.nc')
ddm_model_nt_zblock_ablock_samples = az.from_netcdf(f'hssm_results/{CONDITION}_v~acc_exp-nodetype-nodetype|ppt_a~block-|ppt_z~block-|ppt_t~|ppt.nc')
ddm_model_nt_samples = az.from_netcdf(f'hssm_results/{CONDITION}_v~acc_exp-nodetype-nodetype|ppt_a~|ppt_z~|ppt_t~|ppt.nc')
ddm_model_ntblock_zblock_ablock_samples = az.from_netcdf(f'hssm_results/{CONDITION}_v~acc_exp-block-nodetype-nodetype|ppt_a~block-|ppt_z~block-|ppt_t~|ppt.nc')

#Save in dictionary for arviz comparison
models = {
    'ddm_ntblock_zblock_ablock':ddm_model_ntblock_zblock_ablock_samples, 
    'ddm_nt_zblock_ablock':ddm_model_nt_zblock_ablock_samples, 
    'ddm_model_nt_ablock':ddm_model_nt_ablock_samples, 
    'ddm_ablock':ddm_model_ablock_samples, 
    'ddm_nt': ddm_model_nt_samples
}

#Compare, plot, and save
sns.set_theme('paper')
model_compare = az.compare(models)
az.plot_compare(model_compare)
plt.title(f'Model comparison \n higher is better \n {CONDITION}')
plt.savefig(f'figures/model_comparison_{CONDITION}', dpi = 300)
model_compare.iloc[:, [0, 1, 2, 4]].to_latex(f'model_results/stats/ddm_compare_{CONDITION}.txt', float_format="%.2f")
# STRUCTURED_COMPARE